In [1]:
import numpy as np 
import pandas as pd
import plotly.express as px
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split 


In [2]:
# Leser inn trafikkdata
trafikk_df = pd.read_csv("raw_data/trafikkdata.csv", sep = '[;,|]')#, nrows = 1000)
#trafikk_df#.head()

# viktig: dato, fra tidspunkt(alltid en time), trafikkmengde, felt(bare totalt), dekningsgrad?
# ikke viktig: navn(likt på alle), trafikkregistreringspunkt, veireferanse, fra, til, til tidspunkt
print(trafikk_df)

/tmp/ipykernel_45809/1838320970.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  trafikk_df = pd.read_csv("raw_data/trafikkdata.csv", sep = '[;,|]')#, nrows = 1000)


       Trafikkregistreringspunkt                     Navn    Vegreferanse  \
0                  17510B2483952  Gamle Nygårdsbru sykkel  KV256 S2D1 m75   
1                  17510B2483952  Gamle Nygårdsbru sykkel  KV256 S2D1 m75   
2                  17510B2483952  Gamle Nygårdsbru sykkel  KV256 S2D1 m75   
3                  17510B2483952  Gamle Nygårdsbru sykkel  KV256 S2D1 m75   
4                  17510B2483952  Gamle Nygårdsbru sykkel  KV256 S2D1 m75   
...                          ...                      ...             ...   
348635             17510B2483952  Gamle Nygårdsbru sykkel  KV256 S2D1 m75   
348636             17510B2483952  Gamle Nygårdsbru sykkel  KV256 S2D1 m75   
348637             17510B2483952  Gamle Nygårdsbru sykkel  KV256 S2D1 m75   
348638             17510B2483952  Gamle Nygårdsbru sykkel  KV256 S2D1 m75   
348639             17510B2483952  Gamle Nygårdsbru sykkel  KV256 S2D1 m75   

                           Fra                     Til        Dato  \
0    

In [3]:
# Vil fjerne kolonner i trafikkdatasettet som ikke er relevante for sykkeltrafikken
    # viktig: dato, fra tidspunkt(alltid en time), trafikkmengde, felt(bare totalt), dekningsgrad?
    # ikke viktig: navn(likt på alle), trafikkregistreringspunkt, veireferanse, frra, til, til tidspunkt, ...
    
# Velger kun ut de kolonnene jeg ønsker å beholde    
#trafikk_df = trafikk_df[['Dato','Fra tidspunkt', 'Felt', 'Trafikkmengde']]#, 'Dekningsgrad (%)']]
#trafikk_df

# I kolonnen 'Felt' ønsker jeg kun å se på 'Totalt'-radene (fjerner de radene som ikke har 'totalt'):
trafikk_df = trafikk_df[trafikk_df['Felt'] == 'Totalt']

#Fjerner 'Felt' -kolonnen:'
trafikk_df = trafikk_df[['Dato','Fra tidspunkt', 'Trafikkmengde']]


#Fjerner ':00' fra 'fra tidspunkt' for å få likt format som tidspunktene til værfilene nedenfor
trafikk_df['Fra tidspunkt'] = trafikk_df['Fra tidspunkt'].str.slice(stop=2)


#Endrer kolonnenavnet 'Fra tidspunkt' til 'tidspunkt'
trafikk_df = trafikk_df.rename(columns={'Fra tidspunkt': 'Tidspunkt'})


print(trafikk_df)

#https://stackoverflow.com/questions/56888607/how-to-use-pandas-to-select-certain-columns-in-csv-file 

              Dato Tidspunkt Trafikkmengde
4       2015-07-16        15             -
9       2015-07-16        16           101
14      2015-07-16        17            79
19      2015-07-16        18            56
24      2015-07-16        19            45
...            ...       ...           ...
348619  2023-07-01        19           NaN
348624  2023-07-01        20           NaN
348629  2023-07-01        21           NaN
348634  2023-07-01        22           NaN
348639  2023-07-01        23           NaN

[69728 rows x 3 columns]


In [4]:
#Vil fjerne rader der trafikkmengde gir 'NaN' (trafikkdata mangler)
trafikk_df = trafikk_df.dropna(subset=['Trafikkmengde'])

#Vil også fjerne '-' fra trafikkmengde:
trafikk_df = trafikk_df[trafikk_df['Trafikkmengde'] != '-']

trafikk_df

,Dato,Tidspunkt,Trafikkmengde
9,2015-07-16,16,101
14,2015-07-16,17,79
19,2015-07-16,18,56
24,2015-07-16,19,45
29,2015-07-16,20,41
...,...,...,...
326784,2022-12-31,19,0
326789,2022-12-31,20,0
326794,2022-12-31,21,3
326799,2022-12-31,22,5


In [5]:
# Leser inn værmelding i Florida for hvert år fra 2015 - 2023:
    #Har værfiler helt tilbake til 2011, men kun trafikkdata fra 2015. Dropper dermed å se på værdata fra 2011-2015. 

vær2015_df = pd.read_csv("raw_data/Florida_2015-01-01_2016-01-01_1654174882.csv")
vær2016_df = pd.read_csv("raw_data/Florida_2016-01-01_2017-01-01_1654174902.csv")
vær2017_df = pd.read_csv("raw_data/Florida_2017-01-01_2018-01-01_1654174925.csv")
vær2018_df = pd.read_csv("raw_data/Florida_2018-01-01_2019-01-01_1654175073.csv")
vær2019_df = pd.read_csv("raw_data/Florida_2019-01-01_2020-01-01_1654174955.csv")
vær2020_df = pd.read_csv("raw_data/Florida_2020-01-01_2021-01-01_1654174973.csv")
vær2021_df = pd.read_csv("raw_data/Florida_2021-01-01_2022-01-01_1654174989.csv")
vær2022_df = pd.read_csv("raw_data/Florida_2022-01-01_2023-01-01_1688719054.csv")
vær2023_df = pd.read_csv("raw_data/Florida_2023-01-01_2023-07-01_1688719120.csv")

#vær2023_df.iloc[13424:13500]

# viktig: dato, tid, solskinnstid, lufttemperatur, vindstyrke (og retning? og vindkast)
# ikke så viktig: globalstråling, lufttrykk, relativ luftfuktighet (vindkast og retning)


In [6]:
# Samler alle værfilene i ett datasett:
    # Siden det ikke er så mange værfiler gjør jeg dette manuelt
df_merged1 = vær2015_df.merge(vær2016_df, how='outer') #Slår sammen data fra 2015 og 2016
df_merged2 = df_merged1.merge(vær2017_df, how='outer') #Slår sammen data fra 2015 og 2016 med data fra 2017
df_merged3 = df_merged2.merge(vær2018_df, how='outer')
df_merged4 = df_merged3.merge(vær2019_df, how='outer')
df_merged5 = df_merged4.merge(vær2020_df, how='outer')
df_merged6 = df_merged5.merge(vær2021_df, how='outer')
df_merged7 = df_merged6.merge(vær2022_df, how='outer')
vær_merged_df = df_merged7.merge(vær2023_df, how='outer') # data fra 2015 til 2023 samlet 

# fjerner eventuelle duplikate rekker #(har 446754)
vær_merged_df.drop_duplicates()

#dropper alle rader med manglende verdi #(får 439695)
vær_merged_df = vær_merged_df[(vær_merged_df != 9999.99).all(axis=1)]

#vær_merged_df
    
# https://www.askpython.com/python-modules/pandas/combine-csv-files-using-python

In [7]:
# Vil fjerne kolonner i det samlede datasettet med værfiler som ikke er relevante for sykkeltrafikken
    # viktig: dato, tid, solskinnstid, lufttemperatur, vindstyrke (og retning? og vindkast)
    # ikke så viktig: globalstråling, lufttrykk, relativ luftfuktighet (vindkast og retning)
            
#columns_to_remove_vær = ['Globalstraling', 'Relativ luftfuktighet', 'Vindretning', 'Lufttrykk', 'Vindkast']
columns_to_remove_vær = ['Globalstraling', 'Relativ luftfuktighet', 'Vindretning', 'Lufttrykk']
vær_merged_df = vær_merged_df.drop(columns=columns_to_remove_vær)
#vær_merged_df

# Fjerner eventuelle duplikate rekker 
vær_merged_df.drop_duplicates()

#vær_merged_df

#df_merged.columns

,Dato,Tid,Solskinstid,Lufttemperatur,Vindstyrke,Vindkast
0,2015-01-01,00:00,0.0,6.6,4.2,NaN
1,2015-01-01,00:10,0.0,6.6,4.0,NaN
2,2015-01-01,00:20,0.0,6.6,3.1,NaN
3,2015-01-01,00:30,0.0,6.6,3.7,NaN
4,2015-01-01,00:40,0.0,6.7,2.9,NaN
...,...,...,...,...,...,...
446749,2023-06-30,23:10,0.0,13.7,2.3,3.6
446750,2023-06-30,23:20,0.0,13.6,1.9,3.3
446751,2023-06-30,23:30,0.0,13.6,1.7,3.0
446752,2023-06-30,23:40,0.0,13.6,1.9,3.3


In [8]:
# Vil fikse værdata slik at den har tidsintervaller på 1 time:

# Sjekker typen data:
#trafikk_df.dtypes
#vær_merged_df.dtypes


# Parse the 'Tid' column as a datetime object and set it as the index
vær_merged_df['Tidspunkt'] = vær_merged_df['Tid'].str.split(':').str[0]


#Lager kolonner numeriske (i tilfelle de ikke allerede er det)
kolonner_numeriske = ['Solskinstid', 'Lufttemperatur', 'Vindstyrke', 'Vindkast']
vær_merged_df[kolonner_numeriske] = vær_merged_df[kolonner_numeriske].apply(pd.to_numeric, errors='coerce')

# group by 'hour' og 'date' og regner ut gjennomsnittet for de valgte kolonnene
#grouped_data1 = vær_merged_df.groupby(['Dato', 'Tidspunkt'])[['Lufttemperatur', 'Vindstyrke', 'Vindkast']].mean().round(1)
grouped_data1 = vær_merged_df.groupby(['Dato', 'Tidspunkt'])[['Lufttemperatur', 'Vindstyrke']].mean().round(1)

# group by 'hour' og 'date' og regner ut summen for den valgte kolonnen
grouped_data2 = vær_merged_df.groupby(['Dato', 'Tidspunkt'])[['Solskinstid']].sum()

# group by 'hour' og 'date' og regner ut max for den valgte kolonnen
grouped_data3 = vær_merged_df.groupby(['Dato', 'Tidspunkt'])[['Vindkast']].max().round(1)

# gir 'vær_merged_df' er nytt navn 'værdata'
#værdata = pd.concat([grouped_data1, grouped_data2], axis=1)
værdata = pd.concat([grouped_data1, grouped_data2, grouped_data3], axis=1)

# resetter indeksen slik at 'dato' og 'hour' er vanlige kolonner
værdata.reset_index(inplace=True)



#grouped_data3#['Solskinstid'].sum()
værdata#[9430:9460]
#værdata['Solskinstid'].sum()
    
    
# Fra nello:    
#vær_merged_df = pd.DataFrame(dict(group = ['Solskinstid', 'Lufttemperatur', 'Vindstyrke', 'Vindkast'],
                                 #summer = ['Solskinnstid'],
                                 #gj_snitt = ['Lufttemperatur', 'Vindstyrke', 'Vindkast']))
#vær_merged_df('group').aggregate({'summer': 'sum', 'gj_snitt': 'mean'})

# vær_merged_df




,Dato,Tidspunkt,Lufttemperatur,Vindstyrke,Solskinstid,Vindkast
0,2015-01-01,00,6.6,3.5,0.0,NaN
1,2015-01-01,01,6.8,3.4,0.0,NaN
2,2015-01-01,02,7.0,3.8,0.0,NaN
3,2015-01-01,03,7.2,4.0,0.0,NaN
4,2015-01-01,04,7.5,5.0,0.0,NaN
...,...,...,...,...,...,...
73396,2023-06-30,19,13.7,2.8,0.0,5.7
73397,2023-06-30,20,13.3,2.3,0.0,4.5
73398,2023-06-30,21,13.5,3.2,0.0,7.2
73399,2023-06-30,22,13.6,2.6,0.0,4.5


In [9]:
# Vil slå sammen trafikkfilen 'trafikk_df' og den samlede værfilen 'værdata':
    # 'værdata' var tidligere 'vær_merged_df'
    
combined_data = pd.merge(trafikk_df, værdata, on = ['Dato', 'Tidspunkt'])
                   
combined_data

#fjerne alt der trafikkmengde mangler 

,Dato,Tidspunkt,Trafikkmengde,Lufttemperatur,Vindstyrke,Solskinstid,Vindkast
0,2015-07-16,16,101,13.7,4.3,48.7,7.8
1,2015-07-16,17,79,13.9,3.9,60.0,7.5
2,2015-07-16,18,56,13.2,4.2,60.0,7.5
3,2015-07-16,19,45,12.7,2.9,60.0,6.9
4,2015-07-16,20,41,12.1,2.5,36.0,5.1
...,...,...,...,...,...,...,...
64843,2022-12-31,19,0,1.8,1.3,0.0,4.2
64844,2022-12-31,20,0,0.7,1.7,0.0,4.5
64845,2022-12-31,21,3,0.5,0.8,0.0,2.1
64846,2022-12-31,22,5,0.5,2.2,0.0,6.3


In [10]:
# Vil legge til en kolonne med hvilken ukedag det er 

# Assuming df is your DataFrame and 'Date' is the name of your date column
combined_data['Dato'] = pd.to_datetime(combined_data['Dato'])  # Convert the 'Date' column to DateTime

# Lager en ny kolonne 'Ukedag' som lagrer ukedagene som int (0 = mandag, 1 = tirsdag, ..., 6 = søndag)
combined_data['Ukedag'] = combined_data['Dato'].dt.weekday

combined_data

,Dato,Tidspunkt,Trafikkmengde,Lufttemperatur,Vindstyrke,Solskinstid,Vindkast,Ukedag
0,2015-07-16,16,101,13.7,4.3,48.7,7.8,3
1,2015-07-16,17,79,13.9,3.9,60.0,7.5,3
2,2015-07-16,18,56,13.2,4.2,60.0,7.5,3
3,2015-07-16,19,45,12.7,2.9,60.0,6.9,3
4,2015-07-16,20,41,12.1,2.5,36.0,5.1,3
...,...,...,...,...,...,...,...,...
64843,2022-12-31,19,0,1.8,1.3,0.0,4.2,5
64844,2022-12-31,20,0,0.7,1.7,0.0,4.5,5
64845,2022-12-31,21,3,0.5,0.8,0.0,2.1,5
64846,2022-12-31,22,5,0.5,2.2,0.0,6.3,5


In [11]:
# Deler opp i trenings-, validerings- og testdata

antall_rader = len(combined_data)

# første 70% av combined_data
treningsdata = combined_data[0:int(0.7*antall_rader)] #(0-45392)

# 70%-85% av combined_data
valideringsdata = combined_data[int(0.7*antall_rader):int(0.85*antall_rader)] #(45393 - 55119)

# 85%-100% av combined_data
testdata = combined_data[int(0.85*antall_rader):] #(55120 - 64847)
